In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yaml
import os

# Updated file path based on the actual structure we found
file_path = '../downloaded_files/run-2025-09-22-135430/rooms-generated-p1/0/'

In [ ]:
import xml.etree.ElementTree as ET


# Read the test.xml file relative to the file_path
xml_file_path = os.path.join(file_path, 'test.xml')

try:
    # Check if file exists
    if not os.path.exists(xml_file_path):
        print(f"❌ test result missing")
    else:
        # Read and parse the XML file
        with open(xml_file_path, 'r', encoding='utf-8') as file:
            xml_content = file.read()
        
        # Parse XML for structure analysis
        try:
            # Parse the XML to extract test information
            root = ET.fromstring(xml_content)
            
            # Extract test suite information
            test_suite = root.attrib
            suite_name = test_suite.get('name', 'Unknown')
            total_tests = int(test_suite.get('tests', 0))
            errors = int(test_suite.get('errors', 0))
            failures = int(test_suite.get('failures', 0))
            execution_time = float(test_suite.get('time', 0))

            if total_tests == 0:
                print(f"❌ Test result does not contain test cases")
            
            elif errors > 0 or failures > 0:
                print(f"❌ Test failed")
                
                # Extract and print failure messages
                for testcase in root.findall('testcase'):
                    # Check for failures
                    failure_element = testcase.find('failure')
                    if failure_element is not None:
                        failure_message = failure_element.get('message', 'No message provided')
                        failure_text = failure_element.text or ''
                        
                        print(f"\n📋 Failure Details:")
                        print(f"Test case: {testcase.get('name', 'Unknown')}")
                        print(f"Class: {testcase.get('classname', 'Unknown')}")
                        print(f"Time: {testcase.get('time', 'Unknown')}s")
                        print(f"Failure message: {failure_message}")
                        if failure_text.strip():
                            print(f"\nFailure details:")
                            print("=" * 80)
                            print(failure_text.strip())
                            print("=" * 80)
                    
                    # Check for errors
                    error_element = testcase.find('error')
                    if error_element is not None:
                        error_message = error_element.get('message', 'No message provided')
                        error_text = error_element.text or ''
                        
                        print(f"\n📋 Error Details:")
                        print(f"Test case: {testcase.get('name', 'Unknown')}")
                        print(f"Class: {testcase.get('classname', 'Unknown')}")
                        print(f"Time: {testcase.get('time', 'Unknown')}s")
                        print(f"Error message: {error_message}")
                        if error_text.strip():
                            print(f"\nError details:")
                            print("=" * 80)
                            print(error_text.strip())
                            print("=" * 80)
            else:
                print(f"✅ Test passed (duration: {execution_time:.2f}s)")
                    
        except ET.ParseError as e:
            print(f"❌ Test result parsing error: {e}")
            print(f"\n📄 Raw file content:")
            print("=" * 80)
            print(xml_content)
            
except FileNotFoundError:
    print(f"❌ File not found: {xml_file_path}")
except Exception as e:
    print(f"❌ Error reading file: {e}")

print()
print("Files:")
if os.path.exists(os.path.join(file_path, 'rosbag2')):
    print(f"✅ rosbag2")
else:
    print(f"❌ rosbag2 missing")

if os.path.exists(os.path.join(file_path, 'capture.mp4')):
    if os.path.getsize(os.path.join(file_path, 'capture.mp4')) > 1000:
        print(f"✅ video capture")
    else:
        print(f"❌ video capture invalid")
else:
    print(f"❌ video capture missing")

In [ ]:
# Read scenario.variant file to get start and goal poses

from analysis_common import get_variant_data
import os

variant_data = get_variant_data(file_path)
robot_start = None
robot_end = None

if variant_data and "start_pose" in variant_data:
    start_pose = variant_data['start_pose'] if "start_pose" in variant_data else None
    goal_poses = variant_data['goal_poses'] if "goal_poses" in variant_data else None

    # Store for use in map visualization
    if start_pose and start_pose.get('position'):
        robot_start = {
            'x': start_pose['position']['x'],
            'y': start_pose['position']['y'], 
            'yaw': start_pose.get('orientation', {}).get('yaw', 0)
        }

    if goal_poses and goal_poses[0].get('position'):
        robot_end = {
            'x': goal_poses[0]['position']['x'],
            'y': goal_poses[0]['position']['y'],
            'yaw': goal_poses[0].get('orientation', {}).get('yaw', 0)
        }

static_objects = variant_data['static_objects'] if "static_objects" in variant_data else []

In [ ]:
from IPython.display import Video

# Video playback for capture.mp4
video_path = os.path.join(file_path, 'capture.mp4')

if os.path.exists(video_path):
    # Check if the video file has content
    video_size = os.path.getsize(video_path)
    
    if video_size > 1000:  # If video file is larger than 1KB
        # Display video using IPython Video widget
        display(Video(filename=video_path, width=800, height=400, html_attributes="controls muted autoplay loop"))

In [ ]:

from analysis_common import get_tf_poses
from math import hypot
import analysis_common.map_visualizer as map_visualizer

poses = get_tf_poses(os.path.join(file_path, 'rosbag2'), 'nav2_turtlebot4_base_link_gt')


map_path = os.path.join(file_path, 'maps')
# Find a single .yaml file in the map_paths directory
map_files = [os.path.join(map_path, f) for f in os.listdir(map_path) if f.endswith('.yaml')]

if len(map_files) > 1:
    print(f"Multiple map files found, using the first one: {map_files[0]}")
map_file = map_files[0] if map_files else None

if map_file:
    robot_path = [(pose['x'], pose['y']) for pose in poses]
    
    if poses:
        last_pose = poses[-1]
        dx = last_pose['x'] - robot_end['x']
        dy = last_pose['y'] - robot_end['y']
        distance_to_goal = hypot(dx, dy)
        print(f"Distance from last pose to goal: {distance_to_goal:.3f} meters, {np.degrees(robot_end['yaw'] - last_pose['yaw']):.1f}°")

    # Create map visualizer instance
    viz = map_visualizer.MapVisualizer()
    

    # Load the map
    if viz.load_map(map_file):
        # Create the figure
        fig, ax = viz.create_figure(figsize=(8, 6))
        
        # Draw the path on the map
        viz.draw_path(robot_path, color='blue', linewidth=1, label='Robot Path')
        
        # Add a robot pose at the start of the path
        if robot_start:
            viz.add_robot_pose(robot_start['x'], robot_start['y'], theta=robot_start['yaw'], color='green', size=0.3)

        if robot_end:
            viz.add_robot_pose(robot_end['x'], robot_end['y'], theta=robot_end['yaw'], color='red', size=0.3)

        # Draw static objects
        object_count = 0
        for obj in static_objects:
            object_count += 1
            if 'spawn_pose' in obj and 'position' in obj['spawn_pose']:
                x = obj['spawn_pose']['position']['x']
                y = obj['spawn_pose']['position']['y']
                yaw = obj['spawn_pose']['orientation']['yaw']
                arguments = obj['xacro_arguments']
                
                # Convert xacro_arguments string to dictionary
                # Example: "width:=0.5, length:=0.5, height:=1.5" -> {'width': 0.5, 'length': 0.5, 'height': 1.5}
                draw_args = {}
                if arguments:
                    for arg in arguments.split(','):
                        arg = arg.strip()
                        if ':=' in arg:
                            key, value = arg.split(':=')
                            key = key.strip()
                            value = value.strip()
                            try:
                                # Try to convert to float
                                draw_args[key] = float(value)
                            except ValueError:
                                # Keep as string if conversion fails
                                draw_args[key] = value
                shape = "box" if 'box' in obj['model'] else "circle"
                viz.draw_obstacle(x, y, draw_args=draw_args, yaw=yaw, shape=shape, color='orange', label=f'Object {object_count}')

        # Show legend
        viz.show_legend()
        
        plt.tight_layout()
        plt.show()
        
        # # Print map information
        # print("\\nMap Information:")
        # bounds = viz.get_map_bounds()
        # print(f"Map bounds: x=[{bounds[0]:.2f}, {bounds[1]:.2f}], y=[{bounds[2]:.2f}, {bounds[3]:.2f}]")
    else:
        print("Failed to load map")
        


In [ ]:
# Read and parse the rosbag2 metadata.yaml file

metadata_path = os.path.join(file_path, 'rosbag2/metadata.yaml')

if os.path.exists(metadata_path):
    
    with open(metadata_path, 'r') as file:
        metadata = yaml.safe_load(file)

    # Extract topics and message counts
    topics_data = []
    for topic_info in metadata['rosbag2_bagfile_information']['topics_with_message_count']:
        topic_name = topic_info['topic_metadata']['name']
        message_count = topic_info['message_count']
        topics_data.append({
            'topic': topic_name,
            'message_count': message_count
        })

    # Create DataFrame for easier manipulation
    df = pd.DataFrame(topics_data)
    df = df.sort_values('message_count', ascending=True)  # Sort for better visualization
else:
    df = pd.DataFrame()


In [ ]:
if not df.empty:
    plt.figure(figsize=(12, 4))

    # Create the horizontal bar chart
    bars = plt.barh(df['topic'], df['message_count'], color='steelblue', alpha=0.7)

    # Customize the plot
    plt.xlabel('Message Count', fontsize=12)
    plt.ylabel('ROS2 Topic', fontsize=12)
    plt.title('Message Count Distribution by ROS2 Topic', fontsize=14, fontweight='bold')

    # Add value labels on the bars
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width + max(df['message_count'])*0.01, bar.get_y() + bar.get_height()/2, 
                f'{int(width)}', ha='left', va='center', fontsize=9)

    # Format y-axis labels to show full topic names
    plt.gca().tick_params(axis='y', labelsize=9)
    plt.tight_layout()

    # Add grid for better readability
    plt.grid(axis='x', alpha=0.3, linestyle='--')

    plt.show()

    print(f"Total topics: {len(df)}")
    print(f"Total messages: {metadata['rosbag2_bagfile_information']['message_count']}")